In [2]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 

In [ ]:
# 제로샷 테스트를 하기 위해 모델을 다운받고, 인퍼런스를 실행합니다. 
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "allganize/Llama-3-Alpha-Ko-8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = ""
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": 'Task:라틴 아메리카/카리브해 지역의 인구가 783(7.5%)가 될 때, 아시아의 인구는 얼마가 될까요?\nSQL Table:CREATE TABLE table_22767 (\n    "Year" real,\n    "World" real,\n    "Asia" text,\n    "Africa" text,\n    "Europe" text,\n    "Latin America/Caribbean" text,\n    "Northern America" text,\n    "Oceania" text\n)\nQuery:'}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


/root/anaconda3/envs/text_to_sql/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.45s/it]


# 제로샷 테스트 

In [2]:
print(response)

To find the population of Asia when the population of Latin America/Caribbean is 783 (7.5% of the total world population), we need to first find the total world population and then calculate the population of Asia.

Assuming the data is stored in a SQL table `table_22767`, we can use the following query to find the total world population and the population of Asia:

```sql
SELECT
    "Year",
    "World",
    "Asia",
    "Africa",
    "Europe",
    "Latin America/Caribbean",
    "Northern America",
    "Oceania"
FROM
    table_22767
WHERE
    "Latin America/Caribbean" = 783;
```

If the data is not available in the table, we need to calculate the total world population and the population of Asia based on the given data. Let's assume the total world population is `P` and the population of Latin America/Caribbean is `L`.

The total world population is the sum of the population of all regions:

`P = Asia + Africa + Europe + Latin America/Caribbean + Northern America + Oceania`

The populat

### 정답 
SELECT "Asia" FROM table_22767 WHERE "Latin America/Caribbean" = \'783 (7.5%)\'

### 생성 결과
SELECT (783 / 0.075) AS World_Population, (0.15 * (783 / 0.075)) AS Asia_Population;

잘못된 결과를 생성하고 있고 우리가 원하지 않는 문장들이 포함되어 있습니다.   
Fine-Tuning을 통해 이러한 문제를 해결해보겠습니다. 

In [3]:
import warnings
warnings.filterwarnings("ignore")

import trl
import torch
import datasets
import transformers

import pandas as pd
from random import randint
from datasets import Dataset, load_dataset

from trl import SFTTrainer, setup_chat_format
from peft import LoraConfig, AutoPeftModelForCausalLM

import wandb
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline)

from huggingface_hub import login

import os
import json
from openai import OpenAI

In [4]:
print(f"PyTorch version       : {torch.__version__}")
print(f"Transformers version  : {transformers.__version__}")
print(f"TRL version           : {trl.__version__}")
print(f"CUDA available        : {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version      : {torch.version.cuda}")

PyTorch version       : 2.5.1+cu124
Transformers version  : 4.47.1
TRL version           : 0.13.0
CUDA available        : True
CUDA version      : 12.4


In [ ]:
login(
  token="Huggingface_Token", # 여기에 토큰 추가 
)

In [6]:
# dataset = datasets.load_dataset("Clinton/text-to-sql-v1")
dataset = datasets.load_dataset("daje/kotext-to-sql-v1")
dataset     

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'response', 'source', 'text', 'ko_instruction'],
        num_rows: 262208
    })
})

In [7]:
def add_length_column(dataset):
    df = dataset.to_pandas()
    df["total_length"] = 0
    for column_name in ["ko_instruction", "input", "response"]:
        num_words = df[column_name].astype(str).str.split().apply(len)
        df["total_length"] += num_words

    return df

df = add_length_column(dataset["train"])

def filter_by_total_length(df, difficulty, number_of_samples, random_state=8888):  # random_state 추가
    if difficulty == "easy":
        return df[df["total_length"].between(10, 100)].sample(n=number_of_samples, random_state=random_state)  # iloc 대신 sample 사용
    elif difficulty == "moderate":
        return df[df["total_length"].between(101, 300)].sample(n=number_of_samples, random_state=random_state)
    elif difficulty == "difficult":
        return df[df["total_length"].between(301, 1000)].sample(n=number_of_samples, random_state=random_state)

print(max(df["total_length"].to_list()), min(df["total_length"].to_list()))

910 13


In [8]:
easy = filter_by_total_length(df, "easy", 10000)
medium = filter_by_total_length(df, "moderate", 10000)
hard = filter_by_total_length(df, "difficult", 2000)
dataset = pd.concat([easy, medium, hard])
dataset = dataset.sample(frac=1, random_state=8888)  # random_state 추가
dataset = Dataset.from_pandas(dataset)
easy.shape, medium.shape, hard.shape, dataset.shape

((5000, 7), (5000, 7), (2000, 7), (12000, 8))

In [9]:
# trl docs에 보면 이와 같은 방식으로 SFT Trainer용 데이터를 만들 수 있습니다.
# docs에서는 eos_token을 별도로 추가하라는 안내는 없지만, 저자는 습관적으로 eos_token을 붙혀줍니다.
def get_chat_format(element):
    system_prompt = (
        "You are a helpful programmer assistant that excels at SQL. "
        "Below are sql tables schemas paired with instruction that describes a task. "
        "Using valid SQLite, write a response that appropriately completes the request for the provided tables."
    )
    user_prompt = "### instruction:{ko_instruction} ### Input:{input} ### response:"
    return {
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt.format_map(element)},
            {"role": "assistant", "content": element["response"]+tokenizer.eos_token},
        ]
    }

tokenizer.padding_side = 'right'                      

def tokenize(element):
    # 참고 코드에서처럼, apply_chat_template 사용
    formatted = tokenizer.apply_chat_template(
        get_chat_format(element)["messages"],  # get_chat_format 반환 형식을 맞춰줍니다.
        tokenize=False
    )
    outputs = tokenizer(formatted)
    return {
        "input_ids": outputs["input_ids"],
        "attention_mask": outputs["attention_mask"],
    }
    
# 데이터를 일괄적으로 대화형식으로 변경합니다.
temp_dataset = dataset.map(get_chat_format, remove_columns=dataset.features, batched=False)

# train과 test 데이터를 0.9와 0.1로 분할합니다.
temp_dataset = temp_dataset.train_test_split(test_size=0.05, seed=42)

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

In [10]:
# 정성적으로 변환되었는지 확인합니다.
temp_dataset["train"], temp_dataset["test"]

(Dataset({
     features: ['messages'],
     num_rows: 11400
 }),
 Dataset({
     features: ['messages'],
     num_rows: 600
 }))

In [11]:
temp_dataset["train"][0]

{'messages': [{'content': 'You are a helpful programmer assistant that excels at SQL. Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables.',
   'role': 'system'},
  {'content': '### instruction:\'어떤 시즌이 유일하게 1라운드였나요?\' ### Input:CREATE TABLE table_204_985 (\n    id number,\n    "season" text,\n    "competition" text,\n    "round" text,\n    "club" text,\n    "home" text,\n    "away" text,\n    "aggregate" text\n) ### response:',
   'role': 'user'},
  {'content': 'SELECT "season" FROM table_204_985 WHERE "round" = \'1\'<|end_of_text|>',
   'role': 'assistant'}]}

In [12]:
temp = tokenizer.apply_chat_template(temp_dataset["train"][0]["messages"], tokenizer=False)
print(tokenizer.decode(temp))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful programmer assistant that excels at SQL. Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables.<|eot_id|><|start_header_id|>user<|end_header_id|>

### instruction:'어떤 시즌이 유일하게 1라운드였나요?' ### Input:CREATE TABLE table_204_985 (
    id number,
    "season" text,
    "competition" text,
    "round" text,
    "club" text,
    "home" text,
    "away" text,
    "aggregate" text
) ### response:<|eot_id|><|start_header_id|>assistant<|end_header_id|>

SELECT "season" FROM table_204_985 WHERE "round" = '1'<|end_of_text|><|eot_id|>


In [13]:
test = "1"

In [14]:
# trl docs에 보면 이와 같은 방식으로 SFT Trainer용 데이터를 만들 수 있습니다.
# docs에서는 eos_token을 별도로 추가하라는 안내는 없지만, 저자는 습관적으로 eos_token을 붙혀줍니다.
def get_chat_format(element):
    system_prompt = (
        "You are a helpful programmer assistant that excels at SQL. "
        "Below are sql tables schemas paired with instruction that describes a task. "
        "Using valid SQLite, write a response that appropriately completes the request for the provided tables."
    )
    user_prompt = "### instruction:{ko_instruction} ### Input:{input} ### response:"
    return {
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt.format_map(element)},
            {"role": "assistant", "content": element["response"]+tokenizer.eos_token},  # 여기 닫는 괄호 추가
        ]
    }

tokenizer.padding_side = 'right'                      

def apply_chat_format(element):
    """
    1) get_chat_format(element)를 호출해서
       messages를 생성한 뒤, Dataset에 저장할 dict로 반환합니다.
    """
    chat_format = get_chat_format(element)  # get_chat_format은 원본 코드 그대로 사용
    return {
        "messages": chat_format["messages"]
    }

def tokenize_messages(element):
    """
    2) apply_chat_template + tokenizer(...)를 통해
       input_ids와 attention_mask를 만들어 반환합니다.
    """
    # 위 단계에서 "messages"가 이미 Dataset에 들어가있다고 가정
    formatted = tokenizer.apply_chat_template(
        element["messages"],  # messages 리스트
        tokenize=False
    )
    outputs = tokenizer(formatted)
    return {
        "input_ids": outputs["input_ids"],
        "attention_mask": outputs["attention_mask"]
    }

# train과 test 데이터를 0.9와 0.1로 분할합니다.
dataset = dataset.map(
    apply_chat_format,
    batched=False,
    remove_columns=dataset.features,  # 원하시면 제거
)
dataset = dataset.train_test_split(test_size=0.05)
dataset["train"].to_json("train_dataset.json", orient="records")
dataset["test"].to_json("test_dataset.json", orient="records")

dataset = dataset.map(
    tokenize_messages,
    batched=False,
    remove_columns=["messages"],  # 이제 messages를 더이상 쓰지 않는다면 제거
)

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Map:   0%|          | 0/11400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

In [15]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 11400
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 600
    })
})

In [16]:
from trl import DataCollatorForCompletionOnlyLM
response_template = "<|start_header_id|>assistant<|end_header_id|>"
# 만약 실제로 줄바꿈까지 포함되어 있다면 \n까지 넣어줘야 합니다.
# response_template = "<|start_header_id|>assistant<|end_header_id|>\n"

response_template_ids = tokenizer.encode(response_template, add_special_tokens=False)
collator = DataCollatorForCompletionOnlyLM(
    response_template_ids,
    tokenizer=tokenizer
)


In [17]:
peft_config = LoraConfig(
        lora_alpha=128,                            
        lora_dropout=0.05,                         # Lora 학습 때 사용할 dropout 확률을 지정합니다. 드롭아웃 확률은 과적합 방지를 위해 학습 중 무작위로 일부 뉴런을 비활성화하는 비율을 지정합니다.
        r=256,                                     # Lora의 저차원 공간의 랭크를 지정합니다. 랭크가 높을수록 모델의 표현력이 증가하지만, 계산 비용도 증가합니다.
        bias="none",                               # Lora 적용 시 바이어스를 사용할지 여부를 설정합니다. "none"으로 설정하면 바이어스를 사용하지 않습니다.
        target_modules=["q_proj", "o_proj",        # Lora를 적용할 모델의 모듈 리스트입니다.
                        "k_proj", "v_proj"
                        "up_proj", "down_proj",
                        "gate_proj",
                        ],
        task_type="CAUSAL_LM",                     # 미세 조정 작업 유형을 CAUSAL_LM으로 지정하여 언어 모델링 작업을 수행합니다.
)


args = TrainingArguments(
    output_dir="llama3-12000_ko", # 모델 저장 및 허브 업로드를 위한 디렉토리 지정 합니다.
    num_train_epochs=1,                   # number of training epochs
    # max_steps=100,                          # 100스텝 동안 훈련 수행합니다.
    per_device_train_batch_size=1,          # 배치 사이즈 설정 합니다.
    gradient_accumulation_steps=2,          # 4스텝마다 역전파 및 가중치 업데이트합니다.
    gradient_checkpointing=False,            # 메모리 절약을 위해 그래디언트 체크포인팅 사용합니다.
    optim="adamw_torch_fused",              # 메모리 효율화할 수 있는 fused AdamW 옵티마이저 사용합니다.
    logging_steps=1000,                       # 10스텝마다 로그 기록합니다.
    save_strategy="steps",                  # 매 에폭마다 체크포인트 저장합니다.
    learning_rate=5e-5,                     # 학습률 2e-4로 설정 (QLoRA 논문 기반)합니다.
    bf16=True,                              # 정밀도 설정으로 학습 속도 향상합니다.
    tf32=True,
    max_grad_norm=0.3,                      # 그래디언트 클리핑 값 0.3으로 설정합니다.
    warmup_ratio=0.03,                      # 워밍업 비율 0.03으로 설정 (QLoRA 논문 기반)합니다.
    lr_scheduler_type="constant",           # 일정한 학습률 스케줄러 사용합니다.
    push_to_hub=False,                       # 훈련된 모델을 Hugging Face Hub에 업로드합니다.
    report_to="wandb",                      # wandb로 매트릭 관찰합니다.
)


trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    peft_config=peft_config,
    tokenizer=tokenizer,
    data_collator=collator,
)


In [18]:
# trainer를 학습합니다.
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: daje0601. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
1000,0.289200
2000,0.204700
3000,0.178600
4000,0.180700
5000,0.175600


TrainOutput(global_step=5700, training_loss=0.2011904880456757, metrics={'train_runtime': 1518.2468, 'train_samples_per_second': 7.509, 'train_steps_per_second': 3.754, 'total_flos': 2.4621672351178752e+17, 'train_loss': 0.2011904880456757, 'epoch': 1.0})

In [19]:
# 메모리 초기화
del model
del trainer
torch.cuda.empty_cache()

In [1]:
from openai import OpenAI

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

In [2]:
from datasets import load_dataset
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
eval_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['messages'],
    num_rows: 600
})

In [3]:
eval_dataset[0]["messages"][:2]

[{'content': 'You are a helpful programmer assistant that excels at SQL. Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables.',
  'role': 'system'},
 {'content': "### instruction:'5 7, 2 6'의 점수를 가진 대회는 무엇인가요? ### Input:CREATE TABLE table_name_6 (\n    tournament VARCHAR,\n    score VARCHAR\n) ### response:",
  'role': 'user'}]

In [4]:
idx = 0
sql_chat_completion = client.chat.completions.create(
    model="lora_adapter1",
    messages=eval_dataset[idx]["messages"][:2],
    temperature=0.1,
    max_tokens=500,
)

In [5]:
eval_dataset[idx]["messages"][2]

{'content': 'SELECT tournament FROM table_name_6 WHERE score = "5–7, 2–6"<|end_of_text|>',
 'role': 'assistant'}

In [6]:
print(sql_chat_completion.choices[0].message.content)

SELECT tournament FROM table_name_6 WHERE score = "5–7, 2–6"


In [7]:
from tqdm.auto import tqdm 
result = [] 
for idx in tqdm(range(len(eval_dataset))):
    sql_chat_completion = client.chat.completions.create(
    model="lora_adapter1",
    messages=eval_dataset[idx]["messages"][:2],
    temperature=0.1,
    max_tokens=500,
    stop=["<|eot_id|>"]
    )
    result.append((eval_dataset[idx]["messages"][2]["content"], sql_chat_completion.choices[0].message.content))

print("완료되었습니다.")

  0%|          | 0/600 [00:00<?, ?it/s]

완료되었습니다.


In [8]:
generated_result = [temp[0].replace("<|end_of_text|>", "").strip() == temp[1].replace("<|end_of_text|>", "").strip() for temp in result]

# Exact Match 기준으로 ACC(정확도)를 구합니다.
accuracy = sum(generated_result)/len(generated_result)
print(f"Accuracy: {accuracy*100:.2f}%")

Accuracy: 39.50%


In [ ]:
result[4]

In [9]:
temp = [] 
for idx in tqdm(range(len(eval_dataset))):
    # 1) system / user 메시지 content를 합침
    #    eval_dataset[idx]["messages"][:2] -> [system, user] 두 개의 메시지
    combined_content = "\n".join(msg["content"] for msg in eval_dataset[idx]["messages"][:2])

    temp.append(combined_content)
    

  0%|          | 0/600 [00:00<?, ?it/s]

In [13]:
combined_list = []

for prompt_str, (gold_answer, generated_answer) in zip(temp, result):
    combined_list.append((prompt_str, gold_answer, generated_answer))

# 이제 combined_list[i] = (프롬프트, 정답, 생성결과)
print(combined_list[0])

("You are a helpful programmer assistant that excels at SQL. Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables.\n### instruction:'5 7, 2 6'의 점수를 가진 대회는 무엇인가요? ### Input:CREATE TABLE table_name_6 (\n    tournament VARCHAR,\n    score VARCHAR\n) ### response:", 'SELECT tournament FROM table_name_6 WHERE score = "5–7, 2–6"<|end_of_text|>', 'SELECT tournament FROM table_name_6 WHERE score = "5–7, 2–6"')


In [14]:
openai_evaluation = combined_list

In [ ]:
import os 
import json
# gpt-4o-mini를 사용해서 문제와 정답과 생성된 결과를 넣고, 같은 쿼리인지 확인
# OpenAI API 키 설정 (환경 변수에서 가져오거나 직접 입력)
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

client = OpenAI()

def one_compare_sql_semantics(problem_description, generated_query, ground_truth_query):
    # ChatGPT에게 물어볼 프롬프트 작성
    prompt = f"""다음 문제와 두 SQL 쿼리가 의미적으로 동일한 결과를 반환하는지 판단해주세요:

    문제 설명: {problem_description}

    생성된 쿼리:
    {generated_query}

    정답 쿼리:
    {ground_truth_query}

    두 쿼리가 문제에 대해 의미적으로 동일한 결과를 반환한다면 1로 대답하고,
    그렇지 않다면 0라고 대답한 후 차이점을 설명해주세요.
    쿼리의 구조나 사용된 함수가 다르더라도 결과가 같다면 의미적으로 동일하다고 판단해주세요."""

    # ChatGPT API 호출
    response = client.chat.completions.create(
        model="gpt-4o",  # 또는 사용 가능한 최신 모델
        messages=[
            {"role": "system", "content": "You are a helpful assistant that compares the semantic meaning of SQL queries in the context of a given problem."},
            {"role": "user", "content": prompt}
        ]
    )

    # ChatGPT의 응답 추출
    answer = response.choices[0].message.content.strip()

    # 결과 처리
    is_correct = 1 if answer.lower().startswith("yes") else 0
    explanation = answer[3:] if is_correct == 1 else answer[2:]

    # JSON 형식으로 결과 반환
    result = {
        "answer": is_correct,
        "explanation": explanation.strip()
    }

    return json.dumps(result, ensure_ascii=False)

# 사용 예시

problem = openai_evaluation[1][0]
truth = openai_evaluation[1][1]
generated = openai_evaluation[1][2]

result = one_compare_sql_semantics(problem, generated, truth)
print(result)

{"answer": 0, "explanation": "두 쿼리는 의미적으로 동일한 결과를 반환하지 않습니다. 생성된 쿼리는 \"robert tower\"라는 건물이 있는 주/지역을 찾고 있으며, 정답 쿼리는 \"lovett tower\"라는 건물이 있는 주/지역을 찾고 있습니다. 문제에서 요구한 건물(\"lovett tower\")명의 철자가 두 쿼리에서 다릅니다. 따라서 두 쿼리는 반환하는 결과가 다를 수 있습니다."}


In [ ]:
import os 
import json
from pathlib import Path
from openai import OpenAI
from pqdm.processes import pqdm

# 폴더 생성
os.makedirs("/workspace/openai_result_llama_600_ko", exist_ok=True)

# OpenAI 설정
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
client = OpenAI()

def compare_sql_semantics(idx):
    save_path = f"/workspace/openai_result_llama_600_ko/result_{idx}.json"
    
    # openai_evaluation 사용 (generated_result 대신)
    item = openai_evaluation[idx]
    problem_description, generated_query, ground_truth_query = item
    
    prompt = f"""다음 문제와 두 SQL 쿼리가 의미적으로 동일한 결과를 반환하는지 판단해주세요:
    문제 설명: {problem_description}
    생성된 쿼리:
    {generated_query}
    정답 쿼리:
    {ground_truth_query}
    두 쿼리가 문제에 대해 의미적으로 동일한 결과를 반환한다면 answer에 "1"라고 대답하고,
    그렇지 않다면 "0"라고 대답한 후 차이점을 explanation에 적으세요.
    쿼리의 구조나 사용된 함수가 다르더라도 결과가 같다면 의미적으로 동일하다고 판단해주세요."""
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            response_format={ "type": "json_object" },
            messages=[
                {"role": "system", "content": """You are a helpful assistant that compares the semantic meaning of SQL queries in the context of a given problem.
                return json format below:
                {
                    "answer": "...",
                    "explanation": "..."
                }
                """},
                {"role": "user", "content": prompt}
            ]
        )
        
        answer = response.choices[0].message.content.strip()
        
        # 파일 저장 시도 시 에러 출력
        try:
            with open(save_path, "w", encoding="utf-8") as f:
                json.dump(answer, f, ensure_ascii=False, indent=4)
            print(f"Successfully saved to {save_path}")
        except Exception as e:
            print(f"Error saving file {save_path}: {str(e)}")
            
        return answer
    except Exception as e:
        print(f"Error processing index {idx}: {str(e)}")
        return None

# 실행
indexed_openai_evaluation = list(range(len(openai_evaluation)))
results = pqdm(indexed_openai_evaluation, compare_sql_semantics, n_jobs=40)

/root/anaconda3/envs/text_to_sql/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
QUEUEING TASKS | : 100%|██████████| 10/10 [00:00<00:00, 317.39it/s]
PROCESSING TASKS | : 100%|██████████| 10/10 [00:00<00:00, 1129.11it/s]
COLLECTING RESULTS | : 100%|██████████| 10/10 [00:00<00:00, 52626.15it/s]


In [20]:
import pandas as pd
json_result = []
for result in results:
    json_result.append(json.loads(result))

df = pd.DataFrame(json_result)

df["answer"] = df["answer"].map(lambda x : int(x))

after_accuracy = df["answer"].sum() / len(df["answer"])
print(f"Accuracy: {after_accuracy*100:.2f}%")

Accuracy: 45.17%
